In [100]:
import os
import pandas as pd
import re
from Scripts.json_to_csv import convert_to_csv

### Reading GPS track data
Read the 13 `vehiclePosition` json files and convert them to csv.


__Assumption__: All the `json` and `csv` files should be inside `data` directory

In [101]:
# Current directory path
root = os.path.realpath(".")
data_dir = os.path.join(root, 'data')

In [4]:
# Get all the files in the root containing the word 'vehiclePosition'
input_files = [os.path.join(data_dir, x) for x in os.listdir(data_dir) if 'vehiclePosition' in x]
# For each input files generate the filename with csv extension
output_files = [x.replace('.json', '.csv') for x in input_files]

In [5]:
for i, file in enumerate(input_files):
    convert_to_csv(file, output_files[i])

Preprocessing  /home/nikola/hack-my-ride/data/vehiclePosition03.json
/home/nikola/hack-my-ride/data/vehiclePosition03.csv generated success!
Preprocessing  /home/nikola/hack-my-ride/data/vehiclePosition02.json
/home/nikola/hack-my-ride/data/vehiclePosition02.csv generated success!
Preprocessing  /home/nikola/hack-my-ride/data/vehiclePosition10.json
/home/nikola/hack-my-ride/data/vehiclePosition10.csv generated success!
Preprocessing  /home/nikola/hack-my-ride/data/vehiclePosition06.json
/home/nikola/hack-my-ride/data/vehiclePosition06.csv generated success!
Preprocessing  /home/nikola/hack-my-ride/data/vehiclePosition11.json
/home/nikola/hack-my-ride/data/vehiclePosition11.csv generated success!
Preprocessing  /home/nikola/hack-my-ride/data/vehiclePosition08.json
/home/nikola/hack-my-ride/data/vehiclePosition08.csv generated success!
Preprocessing  /home/nikola/hack-my-ride/data/vehiclePosition13.json
/home/nikola/hack-my-ride/data/vehiclePosition13.csv generated success!
Preprocessing

### [Optional] Merge all the `csv` files into single file

In [6]:
csvs = [pd.read_csv(f) for f in output_files]
print('Concatenating...')
df_data = pd.concat(csvs, ignore_index=True)
gps_file = os.path.join(data_dir, 'gps_track.csv')
df_data.to_csv(gps_file, index=False)

Concatenating...


## 2) Processing GTFS-Sep23 tabular data

In [103]:
gtfs_dir = os.path.join(data_dir, 'gtfs23Sept')
os.chdir(gtfs_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/home/nikola/hack-my-ride/data/gtfs23Sept/data/gtfs23Sept'

### 2.1) Read data

In [91]:
stops = pd.read_csv('stops.txt')
stop_times = pd.read_csv('stop_times.txt', low_memory=False)
calendar = pd.read_csv('calendar.txt')
calendar_dates = pd.read_csv('calendar_dates.txt')
routes = pd.read_csv('routes.txt')
trips = pd.read_csv('trips.txt')

### 2.2) Preprocessing `stops`

In [9]:
stops.head(5)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,0089,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,13.0
1,0470F,NaN,SIMONIS,NaN,50.863666,4.329612,NaN,NaN,0,NaN
2,0471,NaN,SIMONIS,NaN,50.863732,4.329236,NaN,NaN,0,39.0
3,0472,NaN,SIMONIS,NaN,50.863543,4.329023,NaN,NaN,0,39.0
4,0473F,NaN,SIMONIS,NaN,50.863418,4.330031,NaN,NaN,0,NaN


Remove the letters from `stop_id`

In [10]:
stops['stop_id'] = stops['stop_id'].map(lambda x: int(re.sub('[^0-9]', '', x)))

In [11]:
stops.size

28660

Check if we have possible duplicates

In [12]:
stops[stops.duplicated(['stop_id'], keep=False)]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
111,1094,NaN,VALIDA,NaN,50.866310,4.303465,NaN,NaN,0,NaN
112,1094,NaN,VALIDA,NaN,50.866292,4.303465,NaN,NaN,0,NaN
182,1199,NaN,GOFFIN,NaN,50.866834,4.308323,NaN,NaN,0,NaN
183,1199,NaN,GOFFIN,NaN,50.866834,4.308323,NaN,NaN,0,NaN
201,1236,NaN,VALIDA,NaN,50.866256,4.303181,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...
2360,6869,NaN,GARE DE JETTE,NaN,50.880128,4.330471,NaN,NaN,0,NaN
2361,6931,NaN,GARE DE SAINT-JOB,NaN,50.794483,4.362255,NaN,NaN,0,NaN
2362,6931,NaN,GARE DE SAINT-JOB,NaN,50.794465,4.362270,NaN,NaN,0,NaN
2364,6956,NaN,GARE DE SAINT-JOB,NaN,50.794411,4.361518,NaN,NaN,0,NaN


In [13]:
print('Stops Before: ', stops.size)
stops.drop_duplicates(inplace=True)
print('Stops After: ', stops.size)


Stops Before:  28660
Stops After:  28320


Remove Redundant Columns from **Stops**

In [14]:
stops = stops[['stop_id','stop_name', 'stop_lat', 'stop_lon']]

### 2.3) Preprocessing `stop_times`

In [92]:
stop_times.head(5)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,113275577237259001,08:52:00,08:52:00,3107,1,0,0
1,113275577237259001,08:53:00,08:53:00,3177,2,0,0
2,113275577237259001,08:54:03,08:54:03,3171,3,0,0
3,113275577237259001,08:55:30,08:55:30,3172,4,0,0
4,113275577237259001,08:56:18,08:56:18,3173,5,0,0


In [93]:
stop_times[stop_times.duplicated(['stop_id'], keep=False)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,113275577237259001,08:52:00,08:52:00,3107,1,0,0
1,113275577237259001,08:53:00,08:53:00,3177,2,0,0
2,113275577237259001,08:54:03,08:54:03,3171,3,0,0
3,113275577237259001,08:55:30,08:55:30,3172,4,0,0
4,113275577237259001,08:56:18,08:56:18,3173,5,0,0
...,...,...,...,...,...,...,...
1544619,113552074238492000,23:55:51,23:55:51,5751,2,0,0
1544620,113552074238492000,23:56:50,23:56:50,5753F,3,0,0
1544621,113552074238492000,23:58:32,23:58:32,5754,4,0,0
1544622,113552074238492000,24:00:00,24:00:00,1258G,5,0,0


In [94]:
stop_times[stop_times['stop_id'].str.contains('[a-zA-Z]')]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
6,113275577237259001,08:58:00,08:58:00,3175B,7,0,0
17,113275578237259001,08:27:00,08:27:00,3175B,7,0,0
28,113275579237259001,08:20:00,08:20:00,3175B,7,0,0
39,113275580237259001,16:16:00,16:16:00,3175B,7,0,0
48,113275581237259001,15:29:00,15:29:00,3175B,7,0,0
...,...,...,...,...,...,...,...
1544615,113552071238492000,21:55:31,21:55:31,2671F,14,0,0
1544618,113552074238492000,23:55:00,23:55:00,5732F,1,0,1
1544620,113552074238492000,23:56:50,23:56:50,5753F,3,0,0
1544622,113552074238492000,24:00:00,24:00:00,1258G,5,0,0


Let's remove the letter from stops

In [95]:
stop_times['stop_id'] = stop_times['stop_id'].map(lambda x: int(re.sub('[^0-9]', '', x)))

We don't have any duplicates in `stop_times`

In [96]:
print('stop_times Before: ', stop_times.size)
stop_times.drop_duplicates(inplace=True)
print('stop_times After: ', stop_times.size)

stop_times Before:  10812368
stop_times After:  10812368


In [97]:
stop_times['arrival_time'].equals(stop_times['departure_time'])

True

Since the two columns are equal we can safely drop the `departure_time`

In [98]:
stop_times = stop_times[['trip_id','arrival_time','stop_id','stop_sequence']]

Some arrival times exceed 24h so they have to be split into time and exceeding days

In [99]:
def hours_minutes_seconds(td):
    hours = td.seconds//3600
    minutes = (td.seconds//60)%60
    seconds = td.seconds % 60
    return "%d:%02d:%02d" % (hours, minutes, seconds)

stop_times['arrival_time'] = pd.to_timedelta(stop_times['arrival_time'])
stop_times['next_day'] = stop_times['arrival_time'].dt.days
stop_times['arrival_time'] = stop_times['arrival_time'].apply(lambda x: hours_minutes_seconds(x))
stop_times.head(1065)

,trip_id,arrival_time,stop_id,stop_sequence,days
0,113275577237259001,8:52:00,3107,1,0
1,113275577237259001,8:53:00,3177,2,0
2,113275577237259001,8:54:03,3171,3,0
3,113275577237259001,8:55:30,3172,4,0
4,113275577237259001,8:56:18,3173,5,0
...,...,...,...,...,...
1060,113275689237259001,0:50:00,1780,1,1
1061,113275689237259001,0:51:00,6433,2,1
1062,113275689237259001,0:53:00,1262,3,1
1063,113275689237259001,0:54:00,1132,4,1


### 2.4) Preprocessing `calendar`

In [22]:
calendar.head(5)

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,238687005,1,1,1,1,1,0,0,20210920,20211014
1,238689505,0,0,0,0,0,1,0,20210925,20211002
2,238691605,0,0,0,0,0,0,1,20210926,20211010
3,238712503,0,0,0,0,0,1,0,20211009,20211009
4,238683002,1,1,1,1,1,0,0,20211015,20211015


Check if the week day columns are equal in whole dataset

In [23]:
calendar['monday'].equals(calendar['tuesday']) and calendar['monday'].equals(calendar['wednesday']) and calendar['monday'].equals(calendar['thursday']) and calendar['monday'].equals(calendar['friday'])

True

Since all the columns in weekdays are equal we can merge them into single column

In [24]:
calendar['day'] = calendar.apply(lambda x: 0 if x.monday else 1 if x.saturday else 2 if x.sunday else -1 , axis=1)
calendar.drop(['monday','tuesday','wednesday','thursday','friday','saturday','sunday'], axis=1, inplace=True)

Let's make sure we don't have an exceptional value e.g. -1

In [25]:
calendar[calendar['day'] == -1]

,service_id,start_date,end_date,day


In [26]:
calendar.head(5)

,service_id,start_date,end_date,day
0,238687005,20210920,20211014,0
1,238689505,20210925,20211002,1
2,238691605,20210926,20211010,2
3,238712503,20211009,20211009,1
4,238683002,20211015,20211015,0


### 2.5) Preprocessing `routes`

In [27]:
routes.head(5)

,route_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,1,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,B5378C,FFFFFF
1,4,2,SIMONIS - ELISABETH,NaN,1,NaN,ED6C23,FFFFFF
2,5,3,ESPLANADE - CHURCHILL,NaN,0,NaN,B4BD10,000000
3,8,4,GARE DU NORD - STALLE (P),NaN,0,NaN,EA4F80,000000
4,2,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,F6A90B,FFFFFF


In [28]:
routes = routes[['route_id','route_short_name']]

### 2.6) Preprocessing `trips`

In [29]:
trips.head(5)

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,14,237259001,113275577237259001,JULES BORDET,0,8934823,069b0002
1,14,237259001,113275578237259001,JULES BORDET,0,8934827,069b0002
2,14,237259001,113275579237259001,JULES BORDET,0,8934823,069b0002
3,14,237259001,113275580237259001,BORDET STATION,0,8934822,069b0005
4,14,237259001,113275581237259001,JULES BORDET,0,8934826,069b0002


In [30]:
trips = trips[['route_id','service_id','trip_id','trip_headsign','direction_id']]

### 2.7) Saving the processed GTFS

In [ ]:
stops.to_csv('stops_processed.csv', index=False)
stop_times.to_csv('stop_times_processed.csv', index=False)
calendar.to_csv('calendar_processed.csv', index=False)
calendar_dates.to_csv('calendar_dates_processed.csv', index=False)
routes.to_csv('routes_processed.csv', index=False)
trips.to_csv('trips_processed.csv', index=False)

### 2.8) Merging GTFS 

In [31]:
joined_trips = trips.merge(stop_times, on='trip_id', how='inner')\
                    .merge(calendar, on='service_id', how='inner')\
                    .merge(stops, on='stop_id', how='inner')\
                    .merge(routes, on='route_id', how='inner')

In [32]:
joined_trips.head(5)

,route_id,service_id,trip_id,trip_headsign,direction_id,arrival_time,stop_id,stop_sequence,start_date,end_date,day,stop_name,stop_lat,stop_lon,route_short_name
0,14,237259001,113275577237259001,JULES BORDET,0,08:52:00,3107,1,20210920,20211015,0,SCHAERBEEK GARE,50.877636,4.379054,69
1,14,237259001,113275578237259001,JULES BORDET,0,08:21:00,3107,1,20210920,20211015,0,SCHAERBEEK GARE,50.877636,4.379054,69
2,14,237259001,113275579237259001,JULES BORDET,0,08:14:00,3107,1,20210920,20211015,0,SCHAERBEEK GARE,50.877636,4.379054,69
3,14,237259001,113275580237259001,BORDET STATION,0,16:10:00,3107,1,20210920,20211015,0,SCHAERBEEK GARE,50.877636,4.379054,69
4,14,237259001,113275581237259001,JULES BORDET,0,15:23:00,3107,1,20210920,20211015,0,SCHAERBEEK GARE,50.877636,4.379054,69


In [33]:
joined_trips.to_csv('gtfs_merged.csv', index=False)

## 3) Processing GTFS-Sep3 tabular data

Changing the directory to gtfs3Sept and repeating all the steps from 2.1 onward

In [57]:
gtfs_dir = os.path.join(data_dir, 'gtfs3Sept')
os.chdir(gtfs_dir)

## 4) Preprocessing `vehiclePosition`

In [45]:
os.chdir(data_dir)
gps_track = pd.read_csv('gps_track.csv', low_memory=False)

In [46]:
gps_track.head(3)

,time,lineId,directionId,distanceFromPoint,pointId
0,1631177627260,1.0,8161.0,0.0,8012.0
1,1631177627260,1.0,8162.0,0.0,8162.0
2,1631177627260,1.0,8161.0,0.0,8733.0


### 3.1) Preprocessing `lineId` column

Let's proceed by looking at missing values namely `NA` in `lineId` field

In [47]:
line_nas = gps_track.lineId.isna().sum()
total_lines = gps_track.size
"Nulls: {}, Total: {}, Percentage: {}".format(line_nas, total_lines, line_nas/total_lines)

'Nulls: 52831, Total: 97373570, Percentage: 0.0005425599574915452'

In [48]:
gps_track.isnull().sum()

time                     0
lineId               52831
directionId          52831
distanceFromPoint    52831
pointId              52831
dtype: int64

In [49]:
before = gps_track.size
gps_track.dropna(inplace=True)
after = gps_track.size
print("Deleted {}, Before {}, After {}".format(before - after, before, after))

Deleted 264155, Before 97373570, After 97109415


Almost 0.27% of data has been removed

In [50]:
gps_track.rename(columns={'time': 'timestamp'}, inplace=True)

In [51]:
gps_track.head(5)

,timestamp,lineId,directionId,distanceFromPoint,pointId
0,1631177627260,1.0,8161.0,0.0,8012.0
1,1631177627260,1.0,8162.0,0.0,8162.0
2,1631177627260,1.0,8161.0,0.0,8733.0
3,1631177627260,1.0,8162.0,0.0,8072.0
4,1631177627260,1.0,8161.0,0.0,8122.0


In [52]:
gps_track['date'] = pd.to_datetime(gps_track['timestamp'], unit='ms').dt.strftime("%Y%m%d").astype(int)

In [53]:
gps_track['time'] = pd.to_datetime(gps_track['timestamp'], unit='ms').dt.strftime("%H:%M:%S")

In [54]:
weekdays = ['Monday','Tuesday','Wednesday', 'Thursday','Friday']
gps_track['weekday'] = \
    pd.to_datetime(gps_track['timestamp'], unit='ms').dt.strftime("%A") \
    .apply(lambda x: 0 if x in weekdays else 1 if x=='Saturday' else 2 if x=='Sunday' else -1)

In [56]:
gps_track.to_csv('gps_track_processed.csv', index=False)